## TimeShelf: Book Recommendations Across Time and Style

In [1]:
from collections import Counter
import gutenberg.acquire as gacquire
import gutenberg.query as gquery  # print(gquery.list_supported_metadatas())
from hashlib import md5
import imp
import nltk
import os
from os import listdir
from os.path import isdir, isfile
import re
import sys
import zipfile

In [2]:
print(gquery.list_supported_metadatas())

('author', 'formaturi', 'language', 'rights', 'subject', 'title')


# Config

In [3]:
MODULE_EXTENSIONS = ('.py', '.pyc', '.pyo')

In [4]:
ZHOME = '../data/gutenberg_books_en/aleph.gutenberg.org/'

In [5]:
p4_dir_path = '/Users/jmc/git/jaycoskey/Coskey_Metis/projects/Project4/'

In [6]:
GUTENBERG_DOWNLOAD_DIR = p4_dir_path + 'data/gutenberg_books_en/aleph.gutenberg.org/'
# TODO: GUTENBERG_DOWNLOAD_DIR = os.environ['GUTENBERG_DOWNLOAD_DIR']

GUTENBERG_MAX_HEADER_LINES = 300
TITLE_STOPWORDS = """census dictionary encyclo fable gutenberg snate""".split()


# Data Location Info

In [7]:
sample_book_ids = [6001, 7001, 8000, 9043, 10000, 20001, 30000, 40000, 50000]
excluded_book_ids = [89, 3290, 5192, 10681, 13526, 15824, 18251]  # Extra download files
excluded_book_ids.extend([2609, 16762, 11813, 14021, 29274, 35724, 35335])  # Files out of place
excluded_book_ids.extend([1])  # Different format

zip_paths = ['1/0/0/0/10000/10000.zip'
             , '2/0/0/0/20001/20001.zip'
             , '3/0/0/0/30000/30000.zip'
             , '4/0/0/0/40000/40000.zip'
             , '5/0/0/0/50000/50000-0.zip'
             , '6/0/0/6001/6001.zip'
             , '7/0/0/7001/7001-8.zip'
             , '8/0/0/8000/8000-8.zip'
             , '9/0/4/9043/9043.zip'
            ]

# Classes

In [8]:
# class Book():
#    def __init__(self, book_id):
#        self.book_id = book_id
##    
#    def get_data(self):
#        pass
#        # Find file

# Functions

In [9]:
def get_book_content_from_id(id, do_include_content=True):
    book = get_book_from_id(id, do_include_content)
    return ' '.join(book['content_lines'])

In [10]:
def get_book_from_id(id, do_include_content=True):
    book_path_info = get_book_path_info(id)
    book_path = book_path_info[0]
    book = get_book_from_path(id, book_path, do_include_content)
    return book

# New heading

In [11]:
# TODO: Refactor, refactor, refactor
# TODO: Refactor special cases into modular components
def get_book_from_path(id, zpath, do_include_content=True):
    assert(zipfile.is_zipfile(zpath))
    if zpath.endswith('-0.zip'):
        encoding = 'utf-8'
    elif zpath.endswith('-8.zip'):
        encoding = 'latin-1'  # 'iso-8559-1', 'iso8559-1', '8559'
    else:
        encoding = 'ascii'

    line_number = None
    with zipfile.ZipFile(zpath, 'r') as zfile:
        # print('Opened zip file')
        txtfiles = [fname for fname in zfile.namelist() if fname.lower().endswith('.txt')]
        # print(f'txtfiles={txtfiles}')
        if len(txtfiles) != 1:
            print(f'Non-unique file for zpath={zpath}')
            return { 'id': id
                    , 'path': zpath
                    , 'author': None
                    , 'title': None
                    , 'year': None
                    , 'content_lines': None
                    , 'error': 'Non-unique text file'
                   }
        with zfile.open(txtfiles[0], 'r') as tfile:
            # print('Opened book contents within zip file')
            have_seen_start_content_line = False
            have_seen_end_content_line = False
            is_in_header_paragraph = False
            content_start_line_num = 0
            content_end_line_num = sys.maxsize
            author = None
            title = None
            year = 0
            error = ''
            # print(f'type(tfile)={type(tfile)}')
            try:
                lines = [line.decode('utf-8').strip() for line in tfile.readlines()]
            except Exception as ex:
                return { 'id': id
                        , 'path': zpath
                        , 'author': author
                        , 'title': title
                        , 'year': year
                        , 'content_lines': None
                        , 'error': 'Could not decode file'
                       }
            excluded_title_re = re.compile('(census|dictionary|encyclo|fable|gutenberg|senate)')
            gutenberg_re = re.compile('utenberg')
            blank_re = re.compile('^\s*$')
            author_re = re.compile('^Author: (.*)$')
            release_year_re = re.compile('^Release Date: [^,]+, (\d{4})')
            title_re = re.compile('^Title: (.*)$')
            start_content_re = re.compile('^\*\*\*\s*START')
            end_small_print_re = re.compile('^\*END\*THE SMALL PRINT')
            end_content1_re = re.compile('^\*\*\*\s*END')
            end_content2_re = re.compile('^End of the Project')
            
            # Case 0: Year on first row
            year_anywhere_re = re.compile('[^\d](\d{4})[^\d]')
            # Case 1: Year alone on a line with white space (and perhaps a period)
            year_own_line_re = re.compile('^\s*(\d{4})\.?\s*$')
            # Case 2: Year taggled with string: edition, copyright, (c), year, published,
            #                           transcribed, or full month name
            year_tagged_re = re.compile('(copyright|\(c\)|edition|published|transcribed|year|january|february|march|april|may|june|july|august|september|october|november|december)')
            # Case 3: Year within parentheses
            year_parens_re = re.compile('\((\d{4})\)')
            
            # print('About to iterate through book content lines')
            for line_num, line in enumerate(lines):
                line_number = line_num

                if line_num == 0:
                    # Case 0: Year on first row
                    year_anywhere_m = year_anywhere_re.search(line)
                    if year_anywhere_m:
                        year = year_anywhere_m.group(1)

                if line_num < GUTENBERG_MAX_HEADER_LINES:
                    if not have_seen_end_content_line:
                        start_content_m = start_content_re.match(line)
                        if start_content_m:
                            have_seen_start_content_line = True
                            # Start content after START marker
                            content_start_line_num = line_num + 1
                        end_small_print_m = end_small_print_re.match(line)
                        if end_small_print_m:
                            # Start content after end of small print
                            content_start_line_num = line_num + 1
                    if have_seen_start_content_line and gutenberg_re.search(line):
                        is_in_header_paragraph = True
                    if is_in_header_paragraph and blank_re.match(line):
                        is_in_header_paragraph = False
                        # Start content at next non-Gutenberg line
                        content_start_line_num = line_num + 1

                    author_m = author_re.match(line)
                    if author_m:
                        author = author_m.group(1)

                    release_year_m = release_year_re.match(line)
                    if release_year_m:
                        year = release_year_m.group(1)

                    title_m = title_re.match(line)
                    if title_m:
                        title = title_m.group(1)
                        excluded_title_m = excluded_title_re.search(line)
                        if excluded_title_m:
                            return { 'id': id
                                    , 'path': zpath
                                    , 'author': author
                                    , 'title': title
                                    , 'year': year
                                    , 'content_lines':
                                        (lines[content_start_line_num:content_end_line_num]
                                         if do_include_content else None
                                        )
                                    , 'error': f'Excluded due to title {excluded_title_m.group(1)}'
                                }

                    if year == 0:
                        # Case 1: Year alone on a line with white space (and perhaps a period)
                        year_own_line_m = year_own_line_re.match(line)
                        if year_own_line_m:
                            year = year_own_line_m.group(1)
                        else:
                            # Case 2: Year taggled with string
                            year_tagged_m = year_tagged_re.search(line.lower())
                            if year_tagged_m:
                                # Find the year itself
                                year_anywhere_m = year_anywhere_re.search(line)
                                if year_anywhere_m:
                                    year = year_anywhere_m.group(1)
                            else:
                                # Case 3: Year within parentheses
                                year_parens_m = year_parens_re.search(line)
                                if year_parens_m:
                                    year = year_parens_m.group(1)

                end_content1_m = end_content1_re.match(line)
                end_content2_m = end_content2_re.match(line)
                if end_content1_m or end_content2_m:
                    content_end_line_num = line_num
                    have_seen_end_content_line = True
                    # print(f'Found end content tag at line {line_num}; content_end_line_num={content_end_line_num}')
                    break

            # print(f'seen_start={have_seen_start_content_line}; author={author}; title={title}; year={year}')
            if is_in_header_paragraph:
                error += f'{line_number}: Still in header paragraph. '
            if not have_seen_start_content_line:
                error += f'{line_number}: Start content line not found. '
            if not author:
                error += f'{line_number}: Author not found. '
            if not title:
                error += f'{line_number}: Title not found. '
            if year == 0:
                error += f'{line_number}: Year not found. '
            if not have_seen_end_content_line:
                error += f'{line_number}: End content line not found. '
            return { 'id': id
                    , 'path': zpath
                    , 'author': author
                    , 'title': title
                    , 'year': year
                    , 'content_lines':
                        (lines[content_start_line_num:content_end_line_num]
                         if do_include_content else None
                        )
                    , 'error': error
                    }
            # End of zfile context manager block

In [12]:
def get_book_content(book_id):
    book_path_info = get_book_path_info(book_id, base_dir=GUTENBERG_DOWNLOAD_DIR)
    book_path = book_path_info[0]
    book = get_book_from_path(book_id, book_path)
    return book['content_lines']

In [13]:
def get_book_ids(base_dir, excluded_book_ids=[]):
    book_id_strs = []
    for root, dirs, files in os.walk(base_dir):
        if len(files) > 0:
            zip_files = list(filter(lambda fname: fname.endswith('.zip'), files))
            if len(zip_files) > 0:
                book_id_str = zip_files[0]
                hyphen_ind = book_id_str.find('-')
                if hyphen_ind > -1:
                    book_id_str = book_id_str[:hyphen_ind]
                else:
                    zip_ind = book_id_str.find('.zip')
                    book_id_str = book_id_str[:zip_ind]
                if int(book_id_str) in excluded_book_ids:
                    continue
                book_id_strs.append(book_id_str)
    book_ids = [int(id_str) for id_str in book_id_strs]
    return [book_id for book_id in book_ids if book_id not in excluded_book_ids]

In [14]:
def get_book_path_info(id, base_dir=None):
    if not base_dir:
        base_dir = GUTENBERG_DOWNLOAD_DIR
    """Returns (path, error_string)"""
    if id in excluded_book_ids:
        return (None, 'Excluded book id')
    id_str = str(id)
    segments = list(id_str)[:-1]
    if len(segments) == 0:
        segments.insert(0, '0')
    if not base_dir.endswith('/'):
        base_dir = base_dir + '/'
    dir_path = base_dir + '/'.join(segments) + '/' + id_str + '/'
 
    try:
        assert(isdir(dir_path))
    except Exception as ex:
        return (None, 'A=' + str(ex))   
    
    files = listdir(dir_path)

    try: 
        assert(len(files) > 0)
    except Exception as ex:
        return (None, 'B=' + str(ex))

    if len(files) == 1:
        return (dir_path + files[0], '')

    no_hyphens = list(filter(lambda s: not '-' in s, files))
    if len(no_hyphens) > 0:
        return (dir_path + no_hyphens[0], '')

    dash_zeros = list(filter(lambda s: '-0' in s, files))
    if len(dash_zeros) > 0:
        return (dir_path + dash_zeros[0], '')

    dash_eights = list(filter(lambda s: '-8' in s, files))
    if len(dash_eights) > 0:
        return (dir_path + dash_eights[0], '')

    return (None, 'No file with expected filename')

In [15]:
def test_get_book(book_id):
    book_path_info = get_book_path_info(book_id, base_dir=GUTENBERG_DOWNLOAD_DIR)
    if not book_path_info[0]:
        print(f'Book #{book_id}: {book_path_info[1]}')
        return None
    book_path = book_path_info[0]
    book = get_book_from_path(book_id, book_path)
    return book  # print(f"book_id={book_id} has {len(book['content_lines'])} lines")

In [16]:
# Sample use: util_package_contents('nltk')
def util_package_contents(package_name):
    file, pathname, description = imp.find_module(package_name)
    if file:
        raise ImportError('Not a package: %r', package_name)
    # Use a set because some may be both source and compiled.
    return set([os.path.splitext(module)[0]
                for module in os.listdir(pathname)
                if module.endswith(MODULE_EXTENSIONS)
               ])

# Main

In [17]:
valid_book_ids = []
book_errors = {}
for book_id_num, book_id in enumerate(get_book_ids(base_dir=GUTENBERG_DOWNLOAD_DIR)):
    book_path_info = get_book_path_info(book_id, base_dir=GUTENBERG_DOWNLOAD_DIR)
    if not book_path_info[0]:
        print(f'Error with book #{book_id}: {book_path_info[1]}')
        continue
    book_path = book_path_info[0]
    book = get_book_from_path(book_id, book_path)
    if len(book['error']) == 0:
        valid_book_ids.append(book_id)
    else:
        book_errors[book_id] = book['error']
    #if book_id_num > 1000:
    #    break
    # print(f"book_id={book_id} has {len(book['content_lines'])} lines")

Error with book #1: Excluded book id
Error with book #10681: Excluded book id
Non-unique file for zpath=/Users/jmc/git/jaycoskey/Coskey_Metis/projects/Project4/data/gutenberg_books_en/aleph.gutenberg.org/1/0/4/6/10464/10464.zip
Error with book #16762: Excluded book id
Error with book #16762: Excluded book id
Error with book #11813: Excluded book id
Error with book #11813: Excluded book id
Error with book #18251: Excluded book id
Error with book #14021: Excluded book id
Error with book #13526: Excluded book id
Error with book #15824: Excluded book id
Error with book #89: Excluded book id
Error with book #14021: Excluded book id
Error with book #3290: Excluded book id
Error with book #35724: Excluded book id
Error with book #35724: Excluded book id
Error with book #35335: Excluded book id
Error with book #35335: Excluded book id
Error with book #29274: Excluded book id
Error with book #29274: Excluded book id
Error with book #2609: Excluded book id
Error with book #5192: Excluded book id

In [37]:
len(valid_book_ids)

36122

In [41]:
with open('valid_book_ids.txt', 'w') as f:
    for id in sorted(valid_book_ids):
        f.write(str(id) + '\n')

In [42]:
book_errors

{95: 'Could not decode file',
 901: 'Could not decode file',
 906: '1106: Author not found. 1106: Title not found. 1106: End content line not found. ',
 9098: 'Could not decode file',
 9090: 'Could not decode file',
 9095: 'Could not decode file',
 9077: 'Could not decode file',
 9070: 'Could not decode file',
 9073: 'Could not decode file',
 9072: 'Could not decode file',
 9067: 'Could not decode file',
 9050: 'Could not decode file',
 9056: '57: Still in header paragraph. ',
 9051: 'Could not decode file',
 9790: 'Could not decode file',
 9700: 'Could not decode file',
 9658: 'Could not decode file',
 9104: 'Could not decode file',
 9107: 'Could not decode file',
 9106: 'Could not decode file',
 9172: 'Could not decode file',
 915: '14971: Author not found. 14971: Title not found. 14971: End content line not found. ',
 9111: 'Could not decode file',
 9180: 'Could not decode file',
 9184: 'Could not decode file',
 9148: 'Could not decode file',
 9153: 'Could not decode file',
 9878: '

# Latent Dirichlet Allocation

In [43]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [45]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
stop_words.extend("""January February March April May June
    July August September October November December
    year
    about for from into off out over under with
    after before first last then time when which
    all none
    and not
    are become been was were
    can could had may might must should would
    her hers him his its their them they
    one two three four five six seven eight nine ten
    eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty
    than that the there this""".lower().split())
p_stemmer = PorterStemmer()

In [53]:
doc_a = get_book_content_from_id(101)
doc_b = get_book_content_from_id(200)
doc_c = get_book_content_from_id(300)
doc_d = get_book_content_from_id(400)
doc_e = get_book_content_from_id(500)
doc_f = get_book_content_from_id(600)
doc_g = get_book_content_from_id(700)
doc_h = get_book_content_from_id(804)
doc_i = get_book_content_from_id(902)

In [54]:
docs = [doc_a, doc_b, doc_c, doc_d, doc_e, doc_f, doc_g, doc_h, doc_i]
hashes = [ md5(doc.encode('utf-8')).hexdigest() for doc in docs ]

In [55]:
hashes

['d976a5ed600e0778517fe51104bf5448',
 'f3eebae5b23e107cc52f8d938bb87b86',
 '7708de8ddd4dbcd5908c2c15998859b4',
 'dcbc473b4a27ffd89a6aec9187bd0507',
 'de7f0693a4eff456de233071a264905d',
 'e32164c9e514c45fad63fa27404aee54',
 'a8ca01e314f882094ad84395d92a70ce',
 'a98659bc2b30fb9d0d31e0b8f9663fc2',
 '08ecaba88e0b1d373a48e393942df05e']

In [56]:
hash2name = { md5(doc_a.encode('utf-8')).hexdigest(): 'doc_a'
            , md5(doc_b.encode('utf-8')).hexdigest(): 'doc_b'
            , md5(doc_c.encode('utf-8')).hexdigest(): 'doc_c'
            , md5(doc_d.encode('utf-8')).hexdigest(): 'doc_d'
            , md5(doc_e.encode('utf-8')).hexdigest(): 'doc_e'
            , md5(doc_f.encode('utf-8')).hexdigest(): 'doc_f'
            , md5(doc_g.encode('utf-8')).hexdigest(): 'doc_g'
            , md5(doc_h.encode('utf-8')).hexdigest(): 'doc_h'
            , md5(doc_i.encode('utf-8')).hexdigest(): 'doc_i'
            }

In [57]:
len(set(hash2name.keys()))

9

In [61]:
texts = []
for doc in docs:
    # clean and tokenize document string
    raw = doc.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [t for t in tokens if not t in stop_words]
    stemmed_tokens = [p_stemmer.stem(st) for st in stopped_tokens]
    texts.append(stemmed_tokens)

In [62]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=20, id2word = dictionary, passes=20)

In [63]:
ldamodel.print_topics()

[(0,
  '0.001*"great" + 0.001*"c" + 0.001*"gener" + 0.001*"de" + 0.001*"form" + 0.001*"also" + 0.001*"part" + 0.001*"llanbadarn" + 0.001*"state" + 0.001*"use"'),
 (1,
  '0.000*"work" + 0.000*"also" + 0.000*"great" + 0.000*"form" + 0.000*"de" + 0.000*"part" + 0.000*"made" + 0.000*"gener" + 0.000*"c" + 0.000*"name"'),
 (2,
  '0.000*"gener" + 0.000*"use" + 0.000*"made" + 0.000*"form" + 0.000*"de" + 0.000*"also" + 0.000*"name" + 0.000*"found" + 0.000*"see" + 0.000*"state"'),
 (3,
  '0.000*"gener" + 0.000*"great" + 0.000*"de" + 0.000*"form" + 0.000*"state" + 0.000*"work" + 0.000*"c" + 0.000*"name" + 0.000*"1" + 0.000*"also"'),
 (4,
  '0.000*"also" + 0.000*"great" + 0.000*"use" + 0.000*"de" + 0.000*"c" + 0.000*"work" + 0.000*"mani" + 0.000*"made" + 0.000*"name" + 0.000*"state"'),
 (5,
  '0.013*"comput" + 0.007*"hacker" + 0.006*"servic" + 0.005*"system" + 0.005*"board" + 0.005*"peopl" + 0.005*"phone" + 0.004*"like" + 0.003*"call" + 0.003*"public"'),
 (6,
  '0.000*"great" + 0.000*"said" + 0.00

# EDA Code